In [7]:
from data.data_processing import Students
from mdgp import MDGP
import pandas as pd

In [2]:
# Grupos de alunos
students = Students()

# olhar turmas disponíveis
std_classes = students.get_available_classes()

# selecionar uma para usar no MDGP
std_class = std_classes[0]

students_in_class, distance_matrix = students.get_name_distance_matrix(std_class)

# dar uma olhada se da pra formar grupos
students_in_class.shape[0]


27

In [19]:
# criar instância do MDGP para resolver o problema passando a matriz de distâncias e o número de grupos
mdgp = MDGP(distance_matrix, 9)

# obter agenda de resfriamento, tem as funções alfa (1a a ser implementada) e as sugeridas nos slides: exponencial, linear e logaritmica
schedules = [
    {'cooling_function': 'alpha', 'cooling_schedule': mdgp.alpha_schedule(10000, 0.01, 100, 0.9)},
    {'cooling_function': 'exponential', 'cooling_schedule': mdgp.exponential_schedule(10000, 1000, 0.5)},
    {'cooling_function': 'linear', 'cooling_schedule': mdgp.linear_schedule(10000, 1000, 0.9)},
    {'cooling_function': 'log', 'cooling_schedule': mdgp.log_schedule(1000, 10000, 0)}
]

# modo de comparar as funções de resfriamento
for schedule_data in schedules:
    best_cost, best = mdgp.simulated_annealing(schedule_data['cooling_schedule'])
    schedule_data['best_cost'] = best_cost
    schedule_data['best_group_division'] = best


In [20]:
# comparando os resultados
results = pd.DataFrame(schedules)
results

,cooling_function,cooling_schedule,best_cost,best_group_division
0,alpha,"[10000, 10000, 10000, 10000, 10000, 10000, 100...",705,"[7, 0, 1, 6, 6, 0, 4, 7, 4, 4, 2, 5, 8, 2, 8, ..."
1,exponential,"[9.332636185032189e-298, 1.8665272370064378e-2...",706,"[5, 6, 1, 2, 4, 0, 7, 8, 4, 0, 8, 4, 3, 2, 6, ..."
2,linear,"[9100.0, 9100.9, 9101.8, 9102.7, 9103.6, 9104....",678,"[1, 2, 7, 0, 4, 6, 5, 1, 8, 5, 0, 7, 5, 8, 7, ..."
3,log,"[14426.950408889634, 9102.392266268373, 7213.4...",681,"[0, 3, 5, 1, 3, 0, 5, 7, 8, 7, 6, 6, 4, 6, 1, ..."


In [7]:
# se quiser monitorar a convergência, fazer um plot dinâmico etc, usar o iterator do simulated annealing, por exemplo:
# usar o iterator faz ficar bem mais lento, podemos colocar pra retornar uma lista na função, mas quero deixar o iterator pra fazer plot dinâmico

for T, state_cost, state, best_cost, best in mdgp.simulated_annealing_iterator(schedules[0]['cooling_schedule']):
        print("---------------------------------")
        print(f'temperatura atual {T}')
        print(f'custo do estado atual {state_cost}')
        print(f'custo do melhor estado {best_cost}')
        print("---------------------------------")

---------------------------------
temperatura atual 10000
custo do estado atual 649
custo do melhor estado 649
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 662
custo do melhor estado 662
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 653
custo do melhor estado 662
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 654
custo do melhor estado 662
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 654
custo do melhor estado 662
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 655
custo do melhor estado 662
---------------------------------
---------------------------------
temperatura atual 10000
custo do estado atual 653
custo do melhor estado 662
-------------------